### Deep Convolutional Network Cascade for Facial Point Detection
with significant modifications for simplification

In [1]:
from keras import backend as K
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Convolution3D, MaxPooling3D
from keras.layers import Input, Dense, Activation, Flatten
from keras.layers import Reshape, Permute, Lambda
from keras.models import Model
import numpy as np

Using TensorFlow backend.


In [2]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [3]:
def show_landmark(face, landmark):
    face_copied = face.copy().astype(np.uint8)
    for l in landmark:
        x = l[0]
        y = l[1]
        xx = int(face.shape[0]*x)
        yy = int(face.shape[1]*y)
        cv2.circle(face_copied, (xx, yy), 2, (255,255,255), -1)
    return face_copied

In [4]:
def break_to_units(lyr, units_y, units_x):
    # break a 2D array into NxN units
    shape = (None, int(lyr.shape[1]), int(lyr.shape[2]))
    lyr = Reshape(
        (units_y, shape[1]//units_y, shape[2], 1)
    )(lyr)
    lyr = Permute((0, 2, 1, 3))(lyr)
    lyr = Reshape(
        (units_x*units_y, shape[2]//units_x, shape[1]//units_y, 1)
    )(lyr)
    #lyr = Permute((0, 2, 1, 3))(lyr)
    return lyr

def recombine_units(lyr, units_y, units_x):
    shape = (None, None, 
             int(lyr.shape[2]), int(lyr.shape[3]))
    lyr = Permute((1, 0, 2, 3))(lyr)
    lyr = Reshape((shape[2], units_y, shape[3]*units_x, 1))(lyr)
    lyr = Permute((1, 0, 2, 3))(lyr)
    lyr = Reshape((shape[2]*units_y, shape[3]*units_x, 1))(lyr)
    return lyr

def abs_layer(lyr):
    lyr = Lambda(lambda x: K.abs(x))(lyr)
    return lyr

In [5]:
def get_C(lyr, kernel_len, number, p, q):
    lyr = break_to_units(lyr, p, q)
    lyr = Convolution3D(
        filters=number, 
        kernel_size=(p*q,kernel_len,kernel_len),
        strides=(1,1,1),
        padding='same'
    )(lyr)
    lyr = recombine_units(lyr, p, q)
    lyr = Activation('tanh')(lyr)
    return lyr

def get_CR(lyr, kernel_len, number, p, q):
    lyr = get_C(lyr, kernel_len, number, p, q)
    lyr = abs_layer(lyr)
    return lyr

def get_MP(lyr, side_len, p, q):
    lyr = MaxPooling2D(
        pool_size=(side_len, side_len)
    )(lyr)
    lyr = break_to_units(lyr, p, q)
    lyr = Convolution3D(
        filters=1,
        kernel_size=(p*q,1,1),
        strides=(1,1,1)
    )(lyr)
    lyr = recombine_units(lyr, p, q)
    return lyr

def get_FC(lyr, size):
    lyr = Dense(size)(lyr)
    lyr = Activation('tanh')(lyr)
    return lyr

In [6]:
def get_f0():
    inp = Input((48, 48, 1))
    lyr = get_CR(inp, 4, 20, 2, 2)
    lyr = get_MP(lyr, 2, 2, 2)
    lyr = get_CR(lyr, 3, 20, 2, 2)
    lyr = get_MP(lyr, 2, 2, 2)
    lyr = get_CR(lyr, 3, 60, 3, 3)
    lyr = get_MP(lyr, 2, 3, 3)
    #lyr = get_CR(lyr, 2, 80, 2, 2)
    lyr = Flatten()(lyr)
    lyr = get_FC(lyr, 120)
    lyr = get_FC(lyr, 10)
    model = Model(inp, lyr)
    return model

In [9]:
train_set_info = 'dataset/trainImageList.txt'
trainfile = open(train_set_info, 'r')
x = []
y = []
for line in trainfile:
    info = line.rstrip().split(' ')
    #print(info)
    im = cv2.imread('dataset/' + info[0].replace('\\', '/'))
    xy = np.array(info[1:5], dtype=np.int)
    trimmed = im[xy[2]:xy[3], xy[0]:xy[1]]
    trimmed = cv2.resize(trimmed, (48, 48))
    trimmed = cv2.cvtColor(trimmed, cv2.COLOR_BGR2GRAY)
    trimmed = np.reshape(trimmed, trimmed.shape + (1,))
    data = np.array(info[5:], dtype=np.float32)
    data = np.reshape(data, (5, 2))
    for i in range(data.shape[0]):
        data[i][0] = (data[i][0] - xy[0])/(xy[1] - xy[0])
        data[i][1] = (data[i][1] - xy[2])/(xy[3] - xy[2])
    data = np.reshape(data, (10,))
    #plt.imshow(cv2.cvtColor(trimmed, cv2.COLOR_BGR2RGB))
    #plt.imshow(cv2.cvtColor(show_landmark(trimmed, data), cv2.COLOR_BGR2RGB))
    x.append(trimmed)
    y.append(data)
x = np.array(x, dtype=np.float32)
y = np.array(y, dtype=np.float32)

In [11]:
model = get_f0()
model.summary()
model.compile('sgd', 'mse')
model.fit(x, y, epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 48, 48, 1)         0         
_________________________________________________________________
reshape_49 (Reshape)         (None, 2, 24, 48, 1)      0         
_________________________________________________________________
permute_37 (Permute)         (None, None, 24, 2, 48)   0         
_________________________________________________________________
reshape_50 (Reshape)         (None, 4, 24, 24, 1)      0         
_________________________________________________________________
conv3d_13 (Conv3D)           (None, 4, 24, 24, 20)     1300      
_________________________________________________________________
permute_38 (Permute)         (None, 4, None, 24, 24)   0         
_________________________________________________________________
reshape_51 (Reshape)         (None, 24, 2, 48, 1)      0         
__________

Epoch 1/10


InvalidArgumentError: 0 is duplicated in the input.
	 [[Node: training_1/SGD/gradients/permute_51/transpose_grad/InvertPermutation = InvertPermutation[T=DT_INT32, _class=["loc:@permute_51/transpose"], _device="/job:localhost/replica:0/task:0/cpu:0"](permute_51/transpose/perm)]]

Caused by op u'training_1/SGD/gradients/permute_51/transpose_grad/InvertPermutation', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-package